<a href="https://colab.research.google.com/github/davyandr/pet-projects/blob/main/ab/ab_cr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт

In [1]:
! gdown 1RPxb4ujDeQR4KGc03ajmTnpai5UFSzag

Downloading...
From: https://drive.google.com/uc?id=1RPxb4ujDeQR4KGc03ajmTnpai5UFSzag
To: /content/homepage_actions.xls
100% 398k/398k [00:00<00:00, 3.91MB/s]


https://docs.google.com/spreadsheets/d/1RPxb4ujDeQR4KGc03ajmTnpai5UFSzag/edit?usp=sharing&ouid=113106903298233250692&rtpof=true&sd=true \
https://www.kaggle.com/datasets/samtyagi/audacity-ab-testing

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as ss
import warnings
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline


In [3]:
df = pd.read_csv('/content/homepage_actions.xls')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


# Вычисляем конверсию каждого пользователя из действий

In [4]:
df['view'] = (df.action == 'view') * 1
df['click'] = (df.action == 'click') * 1

In [17]:
df = df.groupby('id').click.sum().reset_index().merge(df[['id', 'group']], on='id')

In [20]:
df.columns = ['id', 'conversion', 'group']

In [21]:
df.head()

,id,conversion,group
0,182988,0,experiment
1,182994,1,control
2,182994,1,control
3,183089,0,control
4,183136,0,experiment


Средняя конверсия контроля и теста

In [26]:
df.groupby('group').conversion.mean()

group
control       0.437148
experiment    0.472987
Name: conversion, dtype: float64

# Воспользуемся Z тестом для пропорций для проверки стат. значимости

H0: conversion(control) = conversion(experiment)\
H0: conversion(control) ≠ conversion(experiment)

In [22]:
from statsmodels.stats.proportion import proportions_ztest

In [32]:
pval = proportions_ztest(df.query('''group == "experiment"''').conversion.sum(), df.query('''group == "experiment"''').conversion.count(), df.query('''group == "control"''').conversion.mean())[1]
print(f'P value = {pval}')

P value = 6.906490118823729e-06


Отклоняем нулевую гипотезу
# Запускаем экспериментальный вариант